<a href="https://colab.research.google.com/github/JYchoi22117/SSHS_RnE_JiyeonCHOI/blob/main/LAMDADE_%EC%84%9C%EC%9A%B8%EA%B3%BC%ED%95%99%EA%B3%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**★ 챗봇 실행 전 환경 설정 ★**



1.   실행 전 런타임 유형 변경 필수
- 런타임 → 런타임 유형 변경 → GPU → 저장
2.   경로 설정을 위한 드라이브 이동 필수
- LAMDADE ▼ → 드라이브에 바로가기 추가 → 내 드라이브 → 바로가기추가
(위 작업이 완료되면 내 드라이브에서 LAMDADE 폴더 확인 가능)

In [ ]:
#모듈 설치 후 런타임 다시 시작

!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers #리트리벌 모델에서도 필요한 모듈
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' #허깅페이스
!pip install git+https://github.com/SKTBrain/KoBERT.git
!pip3 install kobert-transformers
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=408819ba862a915243ebade0ac6db389d118613ee8a45b23a0731433eee2aef2
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

**★ 아래 전체 코드 실행 전 환경 설정 ★**

1. 위 모듈 설치 후 '런타임 다시 시작' 필수


*   런타임 → 런타임 다시 시작 → 아래 셀부터 진행 (위 설치 셀 실행 X)


### LAMDADE 챗봇 세팅

- colab 특성 상 실행 시 모든 라이브러리 및 모델을 새로 다운로드 받아야하기 때문에 시간이 걸릴 수 있습니다. (약 2분 내외)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/LAMDADE

In [ ]:
#위에서 설치 완료

# # 필요한 모듈 및 라이브러리 설치
# # ★코랩 환경에서는 매번 새로 설치 필요
# !pip install transformers

In [ ]:
# 설치된 모듈 및 라이브러리 불러오기

# 기본 라이브러리
import pickle
import csv
import pandas as pd
import numpy as np
import time
import gc
import sys

# 이미지 출력
import matplotlib.pyplot as plt
import matplotlib.image as img

# torch, bert model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertModel, BertConfig, BertTokenizer, AutoModel, AutoTokenizer

# lamdade model 관련
from encoder import PolyEncoder, CrossEncoder
from transform_ver2 import SelectionJoinTransform, SelectionSequentialTransform, SelectionConcatTransform
from chat_function import Poly_function, Cross_function

In [ ]:
# 데이터 불러오기 (167,162문장)
print('1. 후보군.csv 파일 불러오기')
data = pd.read_csv('./data/kcbert_base/kor_candidate.csv', encoding='utf8', index_col=0)

# 필요한 컬럼만 지정해서 불러오기
cand_data = data['0']
cand_data.reset_index(drop=True, inplace = True)
cand_data.head()
print(f'후보군으로 사용하는 데이터의 총 개수는 {len(cand_data)} 문장입니다.')
print()

# 사전 계산 후 저장된 후보군 tensor 값 불러오기
print('2. 사전 저장된 후보군 tensor 값 불러오기')
with open('./data/kcbert_base/kor_base_candidate_emb.pickle', 'rb') as file:
  final_cand_emb = pickle.load(file)
print(f'{final_cand_emb.shape[1]} 문장 로드 완료!')
print()

# 모델 불러오기(Poly Encoder, Cross Encoder)
print('3. 사용하는 모델 불러오기')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # gpu 사용

# Poly Encoder Model 경로
# Poly Encoder spec : pretrained model = kcbert base / m = 64 / negative sampling = 1:15
PATH_P = './model/polyencoder_kcbert_base/poly_64_pytorch_model.pt'

# Cross Encoder Model 경로
# Cross Encoder spec : pretrained model = kcbert base / m = 0 / negative sampling = 최종 수정한 코드 이용해서 학습한 Encoder
PATH_C = './model/crossencoder_kcbert_hardneg/cross_0_pytorch_model.pt'

# ★★사전 학습 모델이 같아도 따로 설정해주지 않으면 결과 값에 서로 영향 줌★★

# Pretrained bert model for Poly Encoder Model
bert_name = 'beomi/kcbert-base'
bert_config = BertConfig.from_pretrained(bert_name)
bert = AutoModel.from_pretrained(bert_name)

# Pretrained bert model for Cross Encoder Model
rerank_name = 'beomi/kcbert-base'
rerank_config = BertConfig.from_pretrained(rerank_name)
rerank_bert = AutoModel.from_pretrained(rerank_name)

# Poly Encoder, Cross Encoder Tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(bert_name)
tokenizer2.add_tokens(['\n'], special_tokens = True)

# Input Embedding
context_transform = SelectionJoinTransform(tokenizer = tokenizer2, max_len = 300) # kcbert max 값 300
response_transform = SelectionSequentialTransform(tokenizer = tokenizer2, max_len = 40)
concat_transform = SelectionConcatTransform(tokenizer = tokenizer2, max_len = 300)

# Load Poly Encoder Model
model = PolyEncoder(bert_config, bert = bert, poly_m = 64)
model.resize_token_embeddings(len(tokenizer2))
model.load_state_dict(torch.load(PATH_P), strict = False)
model.to(device)
model.device
print(f'Poly Encoer Model = {PATH_P} 로드 완료!.')

# Load Cross Encoder Model
rerank_model = CrossEncoder(rerank_config, bert = rerank_bert)
rerank_model.resize_token_embeddings(len(tokenizer2))
rerank_model.load_state_dict(torch.load(PATH_C), strict = False)
rerank_model.to(device)
rerank_model.device
print(f'Cross Encoder Model = {PATH_C} 로드 완료!.')
print()

# Encoder별 함수 불러오기
p_func = Poly_function(model=model, bert=bert, tokenizer = tokenizer2, device = device,
                       context_transform = context_transform, response_transform = response_transform)
c_func = Cross_function(model=rerank_model, bert=rerank_bert, tokenizer = tokenizer2, device = device,
                        concat_transform = concat_transform)

print('실행 준비 완료!')

1. 후보군.csv 파일 불러오기
후보군으로 사용하는 데이터의 총 개수는 167162 문장입니다.

2. 사전 저장된 후보군 tensor 값 불러오기
167162 문장 로드 완료!

3. 사용하는 모델 불러오기


Poly Encoer Model = ./model/polyencoder_kcbert_base/poly_64_pytorch_model.pt 로드 완료!.
Cross Encoder Model = ./model/crossencoder_kcbert_hardneg/cross_0_pytorch_model.pt 로드 완료!.

실행 준비 완료!


### EmoBot

In [ ]:
#모듈 실행

import gluonnlp as nlp
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
#cpu 여부 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

#토크나이저/모델/보캡 설정

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
import pandas as pd

corpus_emotion = pd.read_excel("/content/drive/MyDrive/LAMDADE/한국어_단발성_대화_데이터셋.xlsx")
corpus_emotion

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [ ]:
corpus_emotion['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

train_ratio = 0.8  # 8:2 비율을 나타내는 비율
test_ratio = 0.2 # 테스트 데이터 비율

# 각각의 감정에 대한 인덱스를 추출합니다.
emotion_행복 = corpus_emotion[corpus_emotion['Emotion'] == '행복']
emotion_놀람 = corpus_emotion[corpus_emotion['Emotion'] == '놀람']
emotion_분노 = corpus_emotion[corpus_emotion['Emotion'] == '분노']
emotion_공포 = corpus_emotion[corpus_emotion['Emotion'] == '공포']
emotion_혐오 = corpus_emotion[corpus_emotion['Emotion'] == '혐오']
emotion_슬픔 = corpus_emotion[corpus_emotion['Emotion'] == '슬픔']
emotion_중립 = corpus_emotion[corpus_emotion['Emotion'] == '중립']

train_emotion_행복, test_emotion_행복 = train_test_split(emotion_행복, test_size=test_ratio, random_state=42)
train_emotion_놀람, test_emotion_놀람 = train_test_split(emotion_놀람, test_size=test_ratio, random_state=42)
train_emotion_분노, test_emotion_분노 = train_test_split(emotion_분노, test_size=test_ratio, random_state=42)
train_emotion_공포, test_emotion_공포 = train_test_split(emotion_공포, test_size=test_ratio, random_state=42)
train_emotion_혐오, test_emotion_혐오 = train_test_split(emotion_혐오, test_size=test_ratio, random_state=42)
train_emotion_슬픔, test_emotion_슬픔 = train_test_split(emotion_슬픔, test_size=test_ratio, random_state=42)
train_emotion_중립, test_emotion_중립 = train_test_split(emotion_중립, test_size=test_ratio, random_state=42)

dataset_train = pd.concat([train_emotion_행복, train_emotion_놀람, train_emotion_분노, train_emotion_공포, train_emotion_혐오, train_emotion_슬픔, train_emotion_중립], axis = 0)
dataset_test = pd.concat([test_emotion_행복, test_emotion_놀람, test_emotion_분노, test_emotion_공포, test_emotion_혐오, test_emotion_슬픔, test_emotion_중립], axis = 0)


# 훈련 데이터와 테스트 데이터의 감정 비율을 확인합니다.
print("훈련 데이터의 감정 비율:")
print(dataset_train['Emotion'].value_counts())
print("\n테스트 데이터의 감정 비율:")
print(dataset_test['Emotion'].value_counts())

훈련 데이터의 감정 비율:
행복    4829
놀람    4718
분노    4532
공포    4374
혐오    4343
슬픔    4213
중립    3864
Name: Emotion, dtype: int64

테스트 데이터의 감정 비율:
행복    1208
놀람    1180
분노    1133
공포    1094
혐오    1086
슬픔    1054
중립     966
Name: Emotion, dtype: int64


In [ ]:
# depressed_data.loc[(depressed_data['감정'] == "지속되는우울한기분(우울감)"), '감정'] = 0 와 같은 형태로 라벨링
#'분노', '기쁨', '불안', '당황', '슬픔', '상처' (감성 말뭉치 기준)

dataset_train.loc[(dataset_train['Emotion'] == '행복'), 'Emotion'] = 0
dataset_train.loc[(dataset_train['Emotion'] == '놀람'), 'Emotion'] = 1
dataset_train.loc[(dataset_train['Emotion'] == '분노'), 'Emotion'] = 2
dataset_train.loc[(dataset_train['Emotion'] == '공포'), 'Emotion'] = 3
dataset_train.loc[(dataset_train['Emotion'] == '혐오'), 'Emotion'] = 4
dataset_train.loc[(dataset_train['Emotion'] == '슬픔'), 'Emotion'] = 5
dataset_train.loc[(dataset_train['Emotion'] == '중립'), 'Emotion'] = 6

dataset_test.loc[(dataset_test['Emotion'] == '행복'), 'Emotion'] = 0
dataset_test.loc[(dataset_test['Emotion'] == '놀람'), 'Emotion'] = 1
dataset_test.loc[(dataset_test['Emotion'] == '분노'), 'Emotion'] = 2
dataset_test.loc[(dataset_test['Emotion'] == '공포'), 'Emotion'] = 3
dataset_test.loc[(dataset_test['Emotion'] == '혐오'), 'Emotion'] = 4
dataset_test.loc[(dataset_test['Emotion'] == '슬픔'), 'Emotion'] = 5
dataset_test.loc[(dataset_test['Emotion'] == '중립'), 'Emotion'] = 6

#리스트 생성

dataset_train_list = []
for ques1, label1 in zip(dataset_train['Sentence'], dataset_train['Emotion']):
    data_item1 = []
    data_item1.append(ques1)
    data_item1.append(str(label1))
    dataset_train_list.append(data_item1)

dataset_test_list = []
for ques2, label2 in zip(dataset_test['Sentence'], dataset_test['Emotion']):
    data_item2 = []
    data_item2.append(ques2)
    data_item2.append(str(label2))
    dataset_test_list.append(data_item2)

In [ ]:
print(len(dataset_train_list))
print(len(dataset_test_list))

30873
7721


In [ ]:
#데이터 분할

# from sklearn.model_selection import train_test_split
# dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
dataset_train = dataset_train_list
dataset_test = dataset_test_list

#파라미터 설정

max_len = 64
batch_size = 128 #loss 줄이기 위해 높여보기
warmup_ratio = 0.1
num_epochs = 30 #5로 할 경우 런타임이 끊겨서 3으로 줄여봄
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 #loss 줄이기 위해 낮춰보기

In [ ]:
#bert 데이터셋 만드는 클래스 정의하기

class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0][0]
            text_pair = self.dataset[idx][0][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=False
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }

        item['labels'] = int(self.dataset[idx][1])

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']

    def __len__(self):
        return len(self.dataset)

# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, True)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, True)

# 데이터로더 생성
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#학습 데이터셋 예시 추출해보기

print(train_dataset[0])

#bert 분류하는 클래스 정의하기

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7, #감정 개수에 맞게 변경
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

#모델 정의

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# 아래 코드 해석 못함

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

(array([   2,    3, 1185,    3,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1]), array([64]), array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_loader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-16-480d7e0b8d94>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_loader)):


  0%|          | 0/242 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9951817989349365 train acc 0.1328125


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


epoch 1 batch id 201 loss 1.943437933921814 train acc 0.1517801616915423
epoch 1 train acc 0.15321797520661157


...

epoch 18 batch id 1 loss 1.2365394830703735 train acc 0.5
epoch 18 batch id 201 loss 1.438453197479248 train acc 0.4565842661691542
epoch 18 train acc 0.4551084710743802


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 18 test acc 0.280750199920032


  0%|          | 0/242 [00:00<?, ?it/s]

epoch 19 batch id 1 loss 1.3307948112487793 train acc 0.453125


In [ ]:
import random

#감정 예측하는 함수 정의

def emoticon_predict(predict_sentence):

    emoticon_label = {
        'happy': ['😄', '😃', '😁', '😆', '😉', '😊', '😋'],
        'surprised': ['😲', '😮', '😯'],
        'angry': ['😡', '😠', '🤬', '😤'],
        'fear': ['😦', '😧', '😨'],
        'disgust': ['😫', '😩', '😣'],
        'sad': ['😢', '😞', '😔', '😢', '☹️', '🙁', '🥺', '😭'],
        'neutral': ['😗', '🙂', '😙', '😚']
    }

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, tokenizer, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        emoticon_add = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()


            #분노 기쁨 슬픔만 진행하는 것도 고려해보기
            #에크만 참고해서 7가지로 분류
            if np.argmax(logits) == 0:
                # emotion = "행복이"
                emoticon_select = random.choice(emoticon_label['happy'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 1:
                # emotion = "놀람이"
                emoticon_select = random.choice(emoticon_label['surprised'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 2:
                # emotion = "분노가"
                emoticon_select = random.choice(emoticon_label['angry'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 3:
                # emotion = "공포가"
                emoticon_select = random.choice(emoticon_label['fear'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 4:
                # emotion = "혐오가"
                emoticon_select = random.choice(emoticon_label['disgust'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 5:
                # emotion = "슬픔이"
                emoticon_select = random.choice(emoticon_label['sad'])
                emoticon_add.append(emoticon_select)
            elif np.argmax(logits) == 6:
                # emotion = "중립이"
                emoticon_select = random.choice(emoticon_label['neutral'])
                emoticon_add.append(emoticon_select)

            print(best_response + emoticon_add[0])

### LAMDADE 챗봇 한국어 버전 실행

### EmoBot + Lamdade

In [ ]:
print("안녕하세요. 우울증 상담 챗봇 LAMDADE 입니다.")

fig = plt.figure(figsize=(4, 4))
lamda_img = img.imread('./img/bot.png')
plt.imshow(lamda_img)
plt.axis('off')
plt.show()

print("우울증 진단 및 고민 상담을 도와드리고 있습니다.")
print("하고 싶은 말이 있다면 편하게 말씀해주세요.")
print()
print("만약, 상담을 끝내고 싶다면 '종료'를 입력해주세요.")
print("'종료'이 외의 말을 입력할 경우, 대화를 끝내기 어려울 수 있습니다.")

print("*" * 70)

print()
print('LAMDADE:')
name = str(input("귀하의 성함을 말씀해주세요:"))
print()
print(f'{name}님 안녕하세요?')
print(f'{name}님 고민이 있으신가요?')
print()

turn = 0

# 대화 전체를 기록
all_his = []
# 대화 일부를 기록 (turn 4번을 중심으로 reset)
sub_his = []

while True:
    new_context = []

    # poly encoder를 통해서 후보군 top10을 추출하는 과정
    idx_list = []
    top_cand = []

    print('내담자:')
    user_context = [str(input())]
    if user_context != ['종료']:
        all_his.append(user_context[0])
        sub_his.append(user_context[0])

    word = user_context[0].split(' ')

    if user_context == ['종료']:
        check_end = input(f'{name}님 대화를 종료하길 원하시나요? (네/아니오):')
        if check_end == '네':
            print(f'{name}과의 상담을 종료하겠습니다.')

            time.sleep(1)

            # 전체 대화 종료 후, 전반적인 대화에 대한 총점
            check_quality = input(f'{name}님 상담 점수를 매겨주세요. (아주 적절: 2 / 보통: 1 / 부적절:0): ')
            all_his.insert(0, check_quality)

            print('답변 감사합니다.')
            print(f'{name}과의 상담을 종료하겠습니다.')
            break

    else:
        # 첫 발화가 너무 짧을 경우, 추가 발화 요청
        if turn == 0 and len(word) < 3:
            ambiguity = f'{name}님이 느끼는 기분이나 경험한 상황에 대해 조금 더 자세히 말씀해 주실 수 있나요?'
            ambiguity_save = '현재 느끼는 기분이나 관련 상황에 대해 조금 더 자세히 말씀해 주실 수 있나요?'
            all_his.append(ambiguity_save)
            print(ambiguity)
            print()

        else:
            if turn == 0 and len(word) >= 3:
                start = time.time()
                # 첫 대화는 사용자 발화를 input으로 사용
                user_emb = p_func.ctx_emb(*p_func.input_context(user_context))
            else:
                start = time.time()
                # 두 번째 발화부터 4번의 history를 기억하여 input으로 사용
                user_emb = p_func.ctx_emb(*p_func.input_context(sub_his))

            if user_emb is None:
                print('죄송합니다. 말씀을 이해하지 못했어요. 다시 한 번 말씀해주시겠어요?')
                continue

            final_score = p_func.score(user_emb, final_cand_emb)
            new_score = final_score.sort()

            for i in range(10):
                idx_list.append(int(new_score[1][0][-10:][i]))
            for idx in reversed(idx_list):
                top_cand.append(cand_data[idx])

            # top 10 후보군 저장
            top_cand = pd.Series(top_cand)
            # fine-tuning을 마친 cross encoder를 통해 최종 후보군 도출
            rerank_list = []
            # score 순으로 정렬한 후보군 리스트
            final_response_list = []

            if turn == 0:
                cross_score = c_func.text_emb(*c_func.input_text(user_context, top_cand))
                prob_cross_score = F.softmax(cross_score, dim=0)
            else:
                cross_score = c_func.text_emb(*c_func.input_text(sub_his, top_cand))
                prob_cross_score = F.softmax(cross_score, dim=0)  # 확률값 → sum = 1

            for i in range(len(prob_cross_score)):
                rerank_list.append(prob_cross_score[i].item())

            reversed_idxes = np.argsort(rerank_list)[::-1]

            for i in range(len(prob_cross_score)):
                final_response_list.append(top_cand[reversed_idxes[i]])

            # 중복된 응답을 피하기 위해 세 개 답변을 비교하여 출력

            # 첫번째로 가장 적절한 응답
            first_response = final_response_list[0]
            # 두번째로 가장 적절한 응답
            second_response = final_response_list[1]
            # 세번째로 가장 적절한 응답
            third_response = final_response_list[2]

            # threshold 설정을 통해 모든 후보군에 대해 답변이 적절하지 못할 경우 대처
            if max(rerank_list) < 0.2:
                best_response = '죄송합니다. 말씀을 이해하지 못했어요. 다시 한 번 말씀해주시겠어요?'
            else:
                # 다음 문장으로 가장 적절한 값

                # 첫번째 응답, 두번째 응답이 모두 이전 대화 기록에 있을 경우
                if first_response in all_his and second_response in all_his:
                    best_response = third_response
                # 첫번째 응답이 이전 대화 기록에 있을 경우
                elif turn > 0 and first_response in all_his:
                    best_response = second_response
                # 첫번째, 두번째 응답이 모두 이전 대화 기록에 없을 경우
                else:
                    best_response = first_response

                best_response_emoticon = emoticon_predict(best_response)  # 혜진 추가

                if best_response is not None:
                    print('LAMDADE:')
                    print(best_response_emoticon)  # 최적 후보군
                    # print(best_response)

                turn += 1

                # 대화 저장
                all_his.append(best_response)
                sub_his.append(best_response)

                if turn % 4 == 0:
                    sub_his = []

                end = time.time()
                # print('답변까지', round(end-start, 2), '초 소요')
                print()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



안녕하세요. 우울증 상담 챗봇 LAMDADE 입니다.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-4c9352313648>", line 4, in <cell line: 4>
    lamda_img = img.imread('./img/bot.png')
  File "/usr/local/lib/python3.10/dist-packages/matplotlib/image.py", line 1563, in imread
    with img_open(fname) as image:
  File "/usr/local/lib/python3.10/dist-packages/PIL/ImageFile.py", line 105, in __init__
    self.fp = open(fp, "rb")
OSError: [Errno 107] Transport endpoint is not connected: './img/bot.png'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of t

<Figure size 400x400 with 0 Axes>

### 챗봇과의 대화 저장

In [ ]:
# LAMDADE와의 대화 확인
score_dict = {2: '아주 적절', 1: '보통', 0: '부적절'}
print(f'LAMDADE와의 대화는 {score_dict[int(all_his[0])]} 했어요.')
print()
print(*all_his[1:], sep = '\n') # 대화 내용

# 대화 저장
with open('./chatbot_data/conversation.csv', 'a', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(all_his)